# Setup (virtual environment)

In [2]:
import keras
import pickle 
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from econml.iv.nnet import DeepIV
from keras.models import load_model
%matplotlib inline

from tqdm.notebook import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


- gpu check

In [3]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Temp Functions

In [4]:
from typing import List

def optimize_floats(df: pd.DataFrame) -> pd.DataFrame:
    floats = df.select_dtypes(include=['float64']).columns.tolist()
    df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
    return df

def optimize_ints(df: pd.DataFrame) -> pd.DataFrame:
    ints = df.select_dtypes(include=['int64']).columns.tolist()
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    return df

def optimize_objects(df: pd.DataFrame, datetime_features: List[str]) -> pd.DataFrame:
    for col in df.select_dtypes(include=['object']):
        if col not in datetime_features:
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if float(num_unique_values) / num_total_values < 0.5:
                df[col] = df[col].astype('category')
        else:
            df[col] = pd.to_datetime(df[col])
    return df

def optimize(df: pd.DataFrame, datetime_features: List[str] = []):
    return optimize_floats(optimize_ints(optimize_objects(df, datetime_features)))


In [5]:
def update_dummy_columns(row):
    row['brand_{}'.format(row["brand"])] = 1
    row['yyyymm_{}'.format(row["yyyymm"])] = 1
    row['category_1_{}'.format(row["category_1"])] = 1
    return row

# 1. Load Data

In [6]:
df_model = pd.read_feather('../data/DeepIV v1.0.0.ftr')
df_model = df_model[df_model['itt_hour_ln'].notnull()]

In [7]:
df_model[['premium_perc']].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).T

,count,mean,std,min,5%,25%,50%,75%,95%,max
premium_perc,16242602.0,0.362053,0.994836,-0.938326,-0.383333,-0.090909,0.137466,0.508475,1.671756,40.936714


In [8]:
print(df_model.shape)
display(df_model.head())
display(df_model.tail())

(16242602, 216)


,product_id,txn_id,yyyymm,category_1,option_rename,brand,itt_hour_ln,premium_perc,msrp_dollar_ln,brand_likes_ln,with_release_date,days_since_release_ln,likes_count_cumsum,wish_closest_product,likes_count_cumsum_ln,wish_closest_product_ln,VAE0,VAE1,VAE2,VAE3,VAE4,VAE5,VAE6,VAE7,VAE8,VAE9,VAE10,VAE11,VAE12,VAE13,VAE14,VAE15,VAE16,VAE17,VAE18,VAE19,VAE20,VAE21,VAE22,VAE23,VAE24,VAE25,VAE26,VAE27,VAE28,VAE29,VAE30,VAE31,VAE32,VAE33,VAE34,VAE35,VAE36,VAE37,VAE38,VAE39,VAE40,VAE41,VAE42,VAE43,VAE44,VAE45,VAE46,VAE47,VAE48,VAE49,VAE50,VAE51,VAE52,VAE53,VAE54,VAE55,VAE56,VAE57,VAE58,VAE59,VAE60,VAE61,VAE62,VAE63,VAE64,VAE65,VAE66,VAE67,VAE68,VAE69,VAE70,VAE71,VAE72,VAE73,VAE74,VAE75,VAE76,VAE77,VAE78,VAE79,VAE80,VAE81,VAE82,VAE83,VAE84,VAE85,VAE86,VAE87,VAE88,VAE89,VAE90,VAE91,VAE92,VAE93,VAE94,VAE95,VAE96,VAE97,VAE98,VAE99,VAE100,VAE101,VAE102,VAE103,VAE104,VAE105,VAE106,VAE107,VAE108,VAE109,VAE110,VAE111,VAE112,VAE113,VAE114,VAE115,VAE116,VAE117,VAE118,VAE119,VAE120,VAE121,VAE122,VAE123,VAE124,VAE125,VAE126,VAE127,VAE128,VAE129,VAE130,VAE131,VAE132,VAE133,VAE134,VAE135,VAE136,VAE137,VAE138,VAE139,VAE140,VAE141,VAE142,VAE143,VAE144,VAE145,VAE146,VAE147,VAE148,VAE149,VAE150,VAE151,VAE152,VAE153,VAE154,VAE155,VAE156,VAE157,VAE158,VAE159,VAE160,VAE161,VAE162,VAE163,VAE164,VAE165,VAE166,VAE167,VAE168,VAE169,VAE170,VAE171,VAE172,VAE173,VAE174,VAE175,VAE176,VAE177,VAE178,VAE179,VAE180,VAE181,VAE182,VAE183,VAE184,VAE185,VAE186,VAE187,VAE188,VAE189,VAE190,VAE191,VAE192,VAE193,VAE194,VAE195,VAE196,VAE197,VAE198,VAE199
1,2,6,2022-02,shoes,220,Balenciaga,7.867541,-0.280182,6.73961,8.133294,0,0.0,3160.0,2401,8.058644,7.784057,1.909034,1.498761,0.852352,-2.079113,0.596511,-1.173233,-0.09258,-1.505986,-0.803076,1.625568,-0.388218,-1.36814,-0.713074,-0.598995,-0.763059,0.722075,1.56197,-0.167443,-0.510356,0.411824,-0.78674,1.058891,0.819727,1.654205,1.228623,1.295298,0.565713,1.321426,-0.206592,0.036851,-0.25824,0.8206,-1.357059,-0.836837,-0.234835,1.690609,0.315701,-0.472327,0.294032,-0.761382,-1.538711,0.51054,-0.886558,-0.64972,0.703938,2.082435,-1.149042,0.912763,-0.936128,-0.633942,0.013645,0.504946,-0.461937,1.18207,-0.845137,-0.720633,-1.366348,0.02285,-0.049658,0.081791,-0.076514,1.790121,-1.206203,1.39288,1.418635,0.829465,2.222723,0.500948,0.318457,-0.190975,-1.063281,1.245153,-0.198917,-0.109146,0.037992,0.403037,0.586716,-0.659165,-2.141363,-0.776028,0.052681,-0.303432,-1.323512,-0.596104,0.537796,0.523864,1.102149,0.06885,0.351943,-0.521806,-1.030102,0.556163,-0.344134,-0.78897,1.275853,0.506634,-2.54493,0.466514,0.823212,0.112141,0.653163,0.598018,1.055901,-0.469677,-0.203133,0.749473,0.42323,0.185424,0.248024,1.299487,0.027537,-0.280617,-1.412351,-0.115872,-0.577201,0.500824,1.649592,0.096638,-0.398656,0.553979,-0.020444,0.241225,0.106978,0.764752,1.099623,0.356539,0.740165,0.385727,-0.140186,1.005479,-1.453321,-1.131246,-0.107046,1.609106,0.643213,0.568437,1.120584,0.005626,-1.776236,0.894073,-0.382823,0.313857,0.691024,0.604937,-0.883375,0.925584,1.593854,1.441213,0.249394,-0.22586,-0.716903,0.129188,0.334611,0.715158,-0.479226,1.606193,-2.43025,-0.455249,-1.15476,0.762201,-1.942837,0.221258,0.03359,-0.317799,0.264878,-0.705345,0.097787,-1.132969,-1.570749,1.329263,1.311857,0.04649,-1.551999,0.722095,0.729719,2.023951,0.047395,0.157034,-0.814668,-0.214828,-0.94016,-1.584268,-1.129612,-0.520719,-0.497254,-1.478205,-1.94031,0.138772,0.78278,-0.572941,0.661136,0.686502,0.627824,-0.915571,0.9621,-0.344811,-1.192535,0.401531,0.460216,1.369688
2,2,5,2021-12,shoes,250,Balenciaga,7.334878,-0.362187,6.73961,8.133294,0,0.0,3093.0,2401,8.037220,7.784057,1.909034,1.498761,0.852352,-2.079113,0.596511,-1.173233,-0.09258,-1.505986,-0.803076,1.625568,-0.388218,-1.36814,-0.713074,-0.598995,-0.763059,0.722075,1.56197,-0.167443,-0.510356,0.411824,-0.78674,1.058891,0.819727,1.654205,1.228623,1.295298,0.565713,1.321426,-0.206592,0.036851,-0.25824,0.8206,-1.357059,-0.836837,-0.234835,1.690609,0.315701,-0.472327,0.294032,-0.761382,-1.538711,0.51054,-0.88

,product_id,txn_id,yyyymm,category_1,option_rename,brand,itt_hour_ln,premium_perc,msrp_dollar_ln,brand_likes_ln,with_release_date,days_since_release_ln,likes_count_cumsum,wish_closest_product,likes_count_cumsum_ln,wish_closest_product_ln,VAE0,VAE1,VAE2,VAE3,VAE4,VAE5,VAE6,VAE7,VAE8,VAE9,VAE10,VAE11,VAE12,VAE13,VAE14,VAE15,VAE16,VAE17,VAE18,VAE19,VAE20,VAE21,VAE22,VAE23,VAE24,VAE25,VAE26,VAE27,VAE28,VAE29,VAE30,VAE31,VAE32,VAE33,VAE34,VAE35,VAE36,VAE37,VAE38,VAE39,VAE40,VAE41,VAE42,VAE43,VAE44,VAE45,VAE46,VAE47,VAE48,VAE49,VAE50,VAE51,VAE52,VAE53,VAE54,VAE55,VAE56,VAE57,VAE58,VAE59,VAE60,VAE61,VAE62,VAE63,VAE64,VAE65,VAE66,VAE67,VAE68,VAE69,VAE70,VAE71,VAE72,VAE73,VAE74,VAE75,VAE76,VAE77,VAE78,VAE79,VAE80,VAE81,VAE82,VAE83,VAE84,VAE85,VAE86,VAE87,VAE88,VAE89,VAE90,VAE91,VAE92,VAE93,VAE94,VAE95,VAE96,VAE97,VAE98,VAE99,VAE100,VAE101,VAE102,VAE103,VAE104,VAE105,VAE106,VAE107,VAE108,VAE109,VAE110,VAE111,VAE112,VAE113,VAE114,VAE115,VAE116,VAE117,VAE118,VAE119,VAE120,VAE121,VAE122,VAE123,VAE124,VAE125,VAE126,VAE127,VAE128,VAE129,VAE130,VAE131,VAE132,VAE133,VAE134,VAE135,VAE136,VAE137,VAE138,VAE139,VAE140,VAE141,VAE142,VAE143,VAE144,VAE145,VAE146,VAE147,VAE148,VAE149,VAE150,VAE151,VAE152,VAE153,VAE154,VAE155,VAE156,VAE157,VAE158,VAE159,VAE160,VAE161,VAE162,VAE163,VAE164,VAE165,VAE166,VAE167,VAE168,VAE169,VAE170,VAE171,VAE172,VAE173,VAE174,VAE175,VAE176,VAE177,VAE178,VAE179,VAE180,VAE181,VAE182,VAE183,VAE184,VAE185,VAE186,VAE187,VAE188,VAE189,VAE190,VAE191,VAE192,VAE193,VAE194,VAE195,VAE196,VAE197,VAE198,VAE199
16283582,311921,5,2024-07,top,L,Human Made,1.992468,0.559203,4.198936,9.533583,1,1.386294,105.0,81,4.663439,4.406719,2.284745,0.306936,-0.868091,-0.459184,-0.578102,-1.058703,0.916836,-0.029492,0.965827,0.584358,-0.276121,0.314554,-0.459185,-0.195789,-0.180485,2.12471,-0.32637,-0.967598,-1.611056,-0.414427,-0.845065,-0.136836,0.570561,-0.917702,0.032704,-0.17266,-1.588682,-1.528659,-0.465043,1.33398,1.066964,0.125624,-0.735677,0.686294,-0.098865,0.805148,0.29232,0.381034,0.350968,-1.149123,-2.005978,-0.443415,0.671822,-0.075703,-0.773839,-0.825137,-0.569312,-0.968514,0.435879,0.836045,1.389612,0.254575,-1.202764,0.54268,1.204611,-0.806789,0.907811,2.183547,-0.897299,-2.380052,-0.33845,-1.810031,1.659453,1.058408,0.561193,0.535628,-0.488198,0.629268,-0.924544,-1.958857,-1.854309,-0.113677,1.943276,0.246773,-1.143819,0.548756,-0.768095,-0.634324,-1.685012,-1.493768,0.301046,0.124471,-0.122153,-0.96323,0.475242,-2.04938,-0.052405,-0.907777,-0.91787,-1.501978,-0.151971,-0.048061,1.604777,-0.511545,0.222568,0.081712,0.51887,-0.210927,-1.368011,-0.409216,-1.079603,1.452366,0.535651,-0.447672,0.379183,1.033216,0.682557,-0.375063,0.667764,-1.007198,0.194783,0.777102,-0.483,-0.085527,0.59789,-0.234432,-0.823776,-0.058717,-0.520915,0.6315,1.482927,-0.925445,-0.459858,-1.505436,-1.352483,-0.456001,1.268535,-0.630538,1.846922,-0.220236,0.040996,-1.419912,-0.480483,0.693646,0.351148,-0.186405,0.065432,-1.206318,0.760068,-1.485035,-0.940743,-1.28348,-0.559608,0.483735,-0.984608,1.302336,-1.513738,0.976129,0.018083,-0.979743,-0.386676,0.569498,0.576738,-0.412047,1.799549,0.148072,0.922963,0.948913,-0.539899,-0.616029,-0.195734,0.964123,-1.151535,1.299561,-0.54331,0.200305,0.528058,2.089278,-0.475143,1.772738,-0.717116,0.266383,-0.049471,-0.150972,0.259364,2.122253,0.184996,-0.128601,0.08493,0.13626,-0.028448,-0.498802,-1.765108,1.120025,-0.267899,-0.020983,1.206936,0.799843,0.274521,-1.18623,0.804883,-0.284063,-0.653468,-0.683509,-0.174602,0.591751,-0.31483,-0.957431,-0.604849,-0.947908
16283583,311921,4,2024-07,top,XXL,Human Made,1.876960,0.852286,4.198936,9.533583,1,1.386294,105.0,81,4.663439,4.406719,2.284745,0.306936,-0.868091,-0.459184,-0.578102,-1.058703,0.916836,-0.029492,0.965827,0.584358,-0.276121,0.314554,-0.459185,-0.195789,-0.180485,2.12471,-0.32637,-0.967598,-1.611056,-0.414427,-0.845065,-0.136836,0.570561,-0.917702,0.032704,-0.17266,-1.588682,-1.528659,-0.465043,1.33398,1.066964,0.125624,-0.735677,0.686294,-0.098865,0.805148,0.29

## prep data

In [9]:
''' organize product information '''
df_product = df_model.drop_duplicates(subset=['product_id', 'category_1', 'brand'], keep='last').drop(columns=['txn_id', 'likes_count_cumsum_ln', 'wish_closest_product_ln'])
dict_most_sold_option = df_model.groupby(['product_id', 'option_rename'], as_index=False).size().sort_values(by=['product_id', 'size'], ascending=[True, False]).drop_duplicates(subset='product_id', keep='first').set_index('product_id')['option_rename'].to_dict()

In [ ]:
df_model['likes_count_cumsum_1k'] = df_model['likes_count_cumsum'] / 1000

In [10]:
''' oragnize columns and make dummy columns '''
df_model_org = df_model[['product_id', 'itt_hour_ln', 'premium_perc', 'brand', 'category_1', 'yyyymm', 'option_rename', 'msrp_dollar_ln', 'with_release_date', 'days_since_release_ln', 'likes_count_cumsum_1k'] + [col for col in df_model.columns if "VAE" in col]]
df_model_org = optimize(pd.get_dummies(df_model_org, columns=['brand', 'category_1', 'yyyymm', 'option_rename'],  dtype=np.int8))

In [11]:
df_model_org.head()

,product_id,itt_hour_ln,premium_perc,msrp_dollar_ln,with_release_date,days_since_release_ln,likes_count_cumsum_ln,VAE0,VAE1,VAE2,VAE3,VAE4,VAE5,VAE6,VAE7,VAE8,VAE9,VAE10,VAE11,VAE12,VAE13,VAE14,VAE15,VAE16,VAE17,VAE18,VAE19,VAE20,VAE21,VAE22,VAE23,VAE24,VAE25,VAE26,VAE27,VAE28,VAE29,VAE30,VAE31,VAE32,VAE33,VAE34,VAE35,VAE36,VAE37,VAE38,VAE39,VAE40,VAE41,VAE42,VAE43,VAE44,VAE45,VAE46,VAE47,VAE48,VAE49,VAE50,VAE51,VAE52,VAE53,VAE54,VAE55,VAE56,VAE57,VAE58,VAE59,VAE60,VAE61,VAE62,VAE63,VAE64,VAE65,VAE66,VAE67,VAE68,VAE69,VAE70,VAE71,VAE72,VAE73,VAE74,VAE75,VAE76,VAE77,VAE78,VAE79,VAE80,VAE81,VAE82,VAE83,VAE84,VAE85,VAE86,VAE87,VAE88,VAE89,VAE90,VAE91,VAE92,VAE93,VAE94,VAE95,VAE96,VAE97,VAE98,VAE99,VAE100,VAE101,VAE102,VAE103,VAE104,VAE105,VAE106,VAE107,VAE108,VAE109,VAE110,VAE111,VAE112,VAE113,VAE114,VAE115,VAE116,VAE117,VAE118,VAE119,VAE120,VAE121,VAE122,VAE123,VAE124,VAE125,VAE126,VAE127,VAE128,VAE129,VAE130,VAE131,VAE132,VAE133,VAE134,VAE135,VAE136,VAE137,VAE138,VAE139,VAE140,VAE141,VAE142,VAE143,VAE144,VAE145,VAE146,VAE147,VAE148,VAE149,VAE150,VAE151,VAE152,VAE153,VAE154,VAE155,VAE156,VAE157,VAE158,VAE159,VAE160,VAE161,VAE162,VAE163,VAE164,VAE165,VAE166,VAE167,VAE168,VAE169,VAE170,VAE171,VAE172,VAE173,VAE174,VAE175,VAE176,VAE177,VAE178,VAE179,VAE180,VAE181,VAE182,VAE183,VAE184,VAE185,VAE186,VAE187,VAE188,VAE189,VAE190,VAE191,VAE192,VAE193,VAE194,VAE195,VAE196,VAE197,VAE198,VAE199,brand_& Other Stories,brand_1017 Alyx 9SM,brand_47 Brand,brand_A.P.C.,brand_ADSB ANDERSSON BELL,brand_AMI,brand_Acne Studios,brand_Ader Error,brand_Adidas,brand_Aeae,brand_Aime Leon Dore,brand_Alaia,brand_Alden,brand_Alexander McQueen,brand_Alexander Wang,brand_Alo Yoga,brand_Ambush,brand_Amiacalva,brand_And Wander,brand_Andersen Andersen,brand_Anti Social Social Club,brand_Arc'teryx,brand_Arket,brand_Artue,brand_Asics,brand_Athe Vanessabruno Acc,brand_Auralee,brand_Autry,brand_Aviemuah,brand_Axel Arigato,brand_BAPE,brand_BDNS,brand_BDNS-LG TWINS,brand_Balenciaga,brand_Bandai,brand_Bao Bao Issey Miyake,brand_Baracuta,brand_Barbour,brand_Beams,brand_Bensimon,brand_Berluti,brand_Birkenstock,brand_Bohemian Seoul,brand_Bon Ton Toys,brand_Bottega Veneta,brand_Bravest Studios,brand_Buccellati,brand_Bulgari,brand_Burberry,brand_Buttero,brand_C.P. Company,brand_CDG,brand_COS,brand_Cactus Plant Flea Market,brand_California General Store,brand_Calvin Klein,brand_Camper,brand_Canada Goose,brand_Cardinal Red,brand_Carhartt,brand_Carhartt WIP,brand_Carol Shoppe,brand_Cartier,brand_Casio,brand_Castore,brand_Cav Empt,brand_Celine,brand_Champion,brand_Chanel,brand_Charles Jeffrey Loverboy,brand_Chaumet,brand_Chloe,brand_Chrome Hearts,brand_Citizen,brand_Clarks,brand_Coach,brand_Cole Haan,brand_Comfort,brand_Comme des Garcons,brand_Comoli,brand_Converse,brand_Coperni,brand_Corteiz,brand_Cosymosy,brand_Courreges,brand_Crocs,brand_Crystal Haze,brand_Cueren,brand_Dada,brand_Daiwa Pier39,brand_Damiani,brand_Daniel Arsham,brand_Danton,brand_Darkr8m Studio,brand_Deinet,brand_Denim Tears,brand_Destree,brand_Deus Ex Machina,brand_Dickies,brand_Diesel,brand_Dior,brand_Doosan Bears,brand_Dr. Martens,brand_Dragon Diffusion,brand_Drew House,brand_Duvetica,brand_EA7,brand_ERL,brand_ETCE,brand_Ecco,brand_Emis,brand_Engineered Garments,brand_Esenes Worldwide,brand_Essentials,brand_Farfromwhat,brand_Farven,brand_Fear of God,brand_Feathered Friends,brand_Fendi,brand_Ferragamo,brand_Fila,brand_Fjallraven,brand_Forbitches,brand_Fottsfotts,brand_Fragment,brand_Freak's Store,brand_Fullcount,brand_G/Fore,brand_GU,brand_Gallery Dept.,brand_Ganni,brand_Gentle Monster,brand_Givenchy,brand_Glossier,brand_Glowny,brand_Goal Studio,brand_Godashin,brand_Golden Goose,brand_Goyard,brand_Grailz,brand_Gregory,brand_Gucci,brand_Guidi,brand_H&M,brand_HARUTA,brand_HUF,brand_Haag,brand_Hakusan Megane,brand_Hamilton,brand_Helen Kaminski,brand_Helinox,brand_Hereu,brand_Heritage Floss,brand_Hermes,brand_Hidden NY,brand_Hoka,brand_Homme Girls,brand_Homme Plisse Issey Miyake,brand_House of Sunny,brand_Howlin,

# 2. Deep IV (241010 v5.0.0)

In [17]:
treatment_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(848+1,)),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17)])

response_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(848+1,)),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(64, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(32, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(1)])

keras_fit_options = { "epochs": 100,
                      "validation_split": 0.2,
                      "batch_size": 128,
                      "callbacks": [keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]}

deepIvEst = DeepIV(n_components = 10, # number of gaussians in our mixture density network
                   m = lambda z, x : treatment_model(keras.layers.concatenate([z, x])), # treatment model
                   h = lambda t, x : response_model(keras.layers.concatenate([t, x])),  # response model
                   n_samples = 1, # number of samples to use to estimate the response
                   use_upper_bound_loss = False, # whether to use an approximation to the true loss
                   n_gradient_samples = 1, # number of samples to use in second estimate of the response
                                           # (to make loss estimate unbiased)
                   optimizer='sgd', 
                   first_stage_options = keras_fit_options, # options for training treatment model
                   second_stage_options = keras_fit_options) # options for training response model

In [ ]:
y = df_model_org[['itt_hour_ln']].values
t = df_model_org[['premium_perc']].values
x = df_model_org.drop(columns=['product_id', 
                                'itt_hour_ln', # y
                                'premium_perc', # x
                                'likes_count_cumsum_1k', # instrumental variable
                               ]).values
z = df_model_org[['likes_count_cumsum_1k']].values


In [ ]:
''' fit Deep IV by 50 products batch '''
deepIvEst.fit(Y=y, T=t, X=x, Z=z)

In [ ]:
deepIvEst._effect_model.weights

In [ ]:
deepIvEst._effect_model.save("../model/DeepIV_effect_model_241010_v5.h5")

### predict by product

In [ ]:
''' products that used in the model '''
df_product_temp = df_product[df_product['product_id'].isin(df_model_org['product_id'].unique())].drop(columns=['itt_hour_ln', 'premium_perc', 'brand_likes_ln', 'likes_count_cumsum', 'wish_closest_product'])

''' make columns for dummies '''
col_brand = [col for col in df_model_org.columns if 'brand_'in col]
col_cat = [col for col in df_model_org.columns if 'category_1_'in col]
col_yyyymm = [col for col in df_model_org.columns if 'yyyymm_'in col]
col_option = [col for col in df_model_org.columns if 'option_rename_'in col]

df_product_temp[col_brand] = 0
df_product_temp[col_cat] = 0
df_product_temp[col_yyyymm] = 0
df_product_temp[col_option] = 0

''' retain column orders '''
col_order = df_product_temp.columns

''' update brand, category, yyyymm '''
df_product_temp = df_product_temp.progress_apply(update_dummy_columns, axis=1)
df_product_temp = df_product_temp[col_order]
df_product_temp = df_product_temp.set_index('product_id')

''' update option as most sold option '''
for prod in tqdm(df_product_temp.index):
    most_sold_option = dict_most_sold_option[prod]
    df_product_temp.loc[prod ,'option_rename_{}'.format(most_sold_option)] = 1

''' drop unnecessary columns '''
df_product_temp = df_product_temp[[col for col in col_order if col != 'product_id']]
df_product_temp = df_product_temp.drop(columns=['brand', 'category_1', 'yyyymm', 'option_rename'])

''' set as day it is released '''
df_product_temp['days_since_release_ln'] = 0

In [ ]:
dict_results = {}
t_range = np.arange(-0.95, 1.67, 0.01)
for prd in tqdm(df_product_temp.index):
    df_pred = pd.DataFrame([t_range, deepIvEst.predict(t_range, np.tile(df_product_temp.loc[prd], (262, 1)))]).T.rename(columns={0:'premium', 1:'pred_itt_hour_log'})
    df_pred['pred_itt_hour_log'] = df_pred['pred_itt_hour_log'].str[0]
    df_pred['product_id'] = prd
    dict_results[prd] = df_pred
df_pred_results = pd.concat(dict_results.values())

In [ ]:
display(df_pred_results.head())
display(df_pred_results.tail())

In [ ]:
df_pred_results.groupby(['premium'])['pred_itt_hour_log'].mean().plot()

In [ ]:
''' save prediction results '''
df_pred_results.reset_index(drop=True).to_feather("2_analysis/data/3_model/Deep IV/241009 v2/model_prediction_45691prods.ftr")

### load model and replicate

In [ ]:
loaded_effect_model = load_model("2_analysis/data/3_model/Deep IV/241009 v2/DeepIV_batch1000prods_100epochs (_effect_model).h5")

In [ ]:
treatment_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(848+1,)),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(64, activation='relu'),
                                    keras.layers.Dropout(0.17),
                                    keras.layers.Dense(32, activation='relu'),
                                    keras.layers.Dropout(0.17)])

response_model = keras.Sequential([keras.layers.Dense(128, activation='relu', input_shape=(848+1,)),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(64, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(32, activation='relu'),
                                   keras.layers.Dropout(0.17),
                                   keras.layers.Dense(1)])

keras_fit_options = { "epochs": 1,
                      "validation_split": 0.2,
                      "callbacks": [keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]}

model_test = DeepIV(n_components = 10, # number of gaussians in our mixture density network
                   m = lambda z, x : treatment_model(keras.layers.concatenate([z, x])), # treatment model
                   h = lambda t, x : response_model(keras.layers.concatenate([t, x])),  # response model
                   n_samples = 1, # number of samples to use to estimate the response
                   use_upper_bound_loss = False, # whether to use an approximation to the true loss
                   n_gradient_samples = 1, # number of samples to use in second estimate of the response
                                           # (to make loss estimate unbiased)
                   optimizer='adam', 
                   first_stage_options = keras_fit_options, # options for training treatment model
                   second_stage_options = keras_fit_options) # options for training response model

In [ ]:
''' get product ids '''
product_ids = df_model_org['product_id'].unique()

''' shuffle products '''
np.random.seed(42)
np.random.shuffle(product_ids)

''' split all products into 200 products batch '''
product_batch = np.split(product_ids, np.arange(1000, len(product_ids), 1000))

''' slice data that we need '''
df_batch_data = df_model_org[df_model_org['product_id'].isin(product_batch[0])]
y = df_batch_data[['itt_hour_ln']].values
t = df_batch_data[['premium_perc']].values
x = df_batch_data.drop(columns=['product_id', 
                                'itt_hour_ln', # y
                                'premium_perc', # x
                                'likes_count_cumsum_ln', # instrumental variable
                               ]).values
z = df_batch_data[['likes_count_cumsum_ln']].values

''' fit Deep IV by 50 products batch '''
model_test.fit(Y=y, T=t, X=x, Z=z)

In [ ]:
model_test._effect_model = loaded_effect_model

In [ ]:
dict_results = {}
t_range = np.arange(-0.95, 1.67, 0.01)
for prd in tqdm(df_product_temp.index):
    df_pred = pd.DataFrame([t_range, model_test.predict(t_range, np.tile(df_product_temp.loc[prd], (262, 1)))]).T.rename(columns={0:'premium', 1:'pred_itt_hour_log'})
    df_pred['pred_itt_hour_log'] = df_pred['pred_itt_hour_log'].str[0]
    df_pred['product_id'] = prd
    dict_results[prd] = df_pred
df_pred_results = pd.concat(dict_results.values())

In [ ]:
display(df_pred_results.head())
display(df_pred_results.tail())